In [1]:
import torch
from torch import nn
from torch.optim import Adam
device = torch.device('cuda:0')

from sklearn.metrics import r2_score, mean_absolute_error

from models.SCINet import SCINet
from tools.tools import GridSearch

from tools.AirPollution_get_loaders import get_loaders, get_loaders_zero

In [3]:
masked = False  # True or False

if masked:
    train_loader, test_loader = get_loaders_zero(window_size=60, test_size=0.2, batch_size=32)
else:
    train_loader, test_loader = get_loaders(window_size=60, test_size=0.2, batch_size=32)

criterion = nn.MSELoss()
eval_metrics = [r2_score, mean_absolute_error]
lr = 1e-3

100%|██████████| 306/306 [00:00<00:00, 297440.79it/s]


In [ ]:
data_name = 'LG'  # 'LG' or 'AirPollution'
model_name = input('Enter the name of the model: ')

basic_params = {
    'T': 60,
    'n': 7
}

param_grid = {
    'skip_hidden_size': [12, 16, 24, 32],
    'T_modified': [20, 30, 40],
    'skip': [8, 10, 12]
}

model_trainer = GridSearch(criterion, eval_metrics, device,
                           temp_save_path=f'checkpoints/{data_name}/{model_name}_temp.pt')
model2_best = model_trainer.train_by_grid(SCINet, basic_params, param_grid, Adam, train_loader,
                                          test_loader, lr, patience=5, epochs=50, save_filename=f'checkpoints/{model_name}_best.pt')


In [ ]:
model_trainer.test(test_loader)
model_trainer.plot_losses(plot_title=f'{model_name}', save_filename=f'{model_name}_losses.png')